In [6]:
import cv2
import numpy as np
import math


# ----- Helper Functions -----
def compute_marker_angle(marker_corners):
    """
    Compute the marker orientation from its corners.
    We assume the order: top-left, top-right, bottom-right, bottom-left.
    Using the line from top-left to top-right, compute the angle in degrees.
    """
    top_left = marker_corners[0]
    top_right = marker_corners[1]
    angle = math.degrees(math.atan2(top_right[1] - top_left[1], top_right[0] - top_left[0]))
    return angle


def detect_circles(gray_image):
    """
    Use OpenCV's SimpleBlobDetector to detect circles.
    Adjust parameters as necessary based on your image.
    Returns an array of (x,y) coordinates for the detected circles.
    """
    # Set up SimpleBlobDetector parameters.
    params = cv2.SimpleBlobDetector_Params()
    params.filterByArea = True
    params.minArea = 5  # Adjust depending on the expected size of circles
    params.maxArea = 5000  # Adjust as needed
    params.filterByCircularity = True
    params.minCircularity = 0.2  # Tighter circularity
    params.filterByInertia = False
    params.minInertiaRatio = 0.5

    detector = cv2.SimpleBlobDetector_create(params)
    keypoints = detector.detect(gray_image)

    # Draw detected keypoints for visualization (optional)
    im_with_keypoints = cv2.drawKeypoints(gray_image, keypoints, np.array([]), (0, 0, 255),
                                          cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    cv2.imshow("Detected Circles", im_with_keypoints)
    cv2.waitKey(0)
    cv2.destroyWindow("Detected Circles")

    if len(keypoints) == 0:
        return np.empty((0, 2), dtype=float)

    points = np.array([kp.pt for kp in keypoints])
    return points


def fit_line_angle(points):
    """Fit line to points and return angle in degrees."""
    if len(points) < 2:
        return None

    points = points.astype(np.float32)
    [vx, vy, x0, y0] = cv2.fitLine(points, cv2.DIST_L2, 0, 0.01, 0.01)

    # Extract scalar values from the arrays
    vx_scalar = float(vx[0])
    vy_scalar = float(vy[0])

    # Calculate angle in degrees
    angle = math.degrees(math.atan2(vy_scalar, vx_scalar))
    return angle


# ----- Main Script -----
# Load image
print("[INFO] Loading image...")
image_path = "DamasconeA.png"
image = cv2.imread(image_path)

if image is None:
    print(f"[ERROR] Could not read image from {image_path}")
    exit()

# Make a copy for visualization
output = image.copy()

# ------ Step 1: Detect ArUco Marker ------
# Use your working dictionary
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_ARUCO_ORIGINAL)
arucoParams = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(aruco_dict, arucoParams)
(corners, ids, rejected) = detector.detectMarkers(image)

if len(corners) > 0:
    print(f"[INFO] Detected {len(corners)} marker(s) using DICT_ARUCO_ORIGINAL")

    # For demonstration we use the first marker detected
    marker_corners = corners[0][0]  # corners[0] is a list with 4 points
    marker_angle = compute_marker_angle(marker_corners)
    print(f"[INFO] Marker angle: {marker_angle:.2f} degrees")

    # Draw the detected marker for visualization
    cv2.aruco.drawDetectedMarkers(output, corners, ids)
    cv2.imshow("Detected Markers", output)
    cv2.waitKey(0)
    cv2.destroyWindow("Detected Markers")

else:
    print("[ERROR] No ArUco markers detected!")
    exit()

# ------ Step 2: Detect the circles (blobs) -------
# For better blob detection, work in grayscale.
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
points = detect_circles(gray)

if points.shape[0] == 0:
    print("[ERROR] No circles (blobs) detected!")
    exit()

# ------ Step 3: Identify central row/column for the object -------
# Compute median coordinate as rough center
median_x = np.median(points[:, 0])
median_y = np.median(points[:, 1])

# Set a tolerance in pixels. Adjust tolerance based on spacing between circles.
tol = 10  # pixels tolerance

# For the central horizontal line, pick points near median_y
horizontal_points = points[np.abs(points[:, 1] - median_y) < tol]
# For the central vertical line, pick points near median_x
vertical_points = points[np.abs(points[:, 0] - median_x) < tol]

print(
    f"[INFO] Found {len(horizontal_points)} points for the horizontal line and {len(vertical_points)} for the vertical line.")

# ------ Step 4: Fit lines and compute their angles -------
horizontal_angle = fit_line_angle(horizontal_points)
vertical_angle = fit_line_angle(vertical_points)

if horizontal_angle is None or vertical_angle is None:
    print("[ERROR] Not enough points to compute line angles.")
    exit()

print(f"[INFO] Horizontal line angle: {horizontal_angle:.2f} degrees")
print(f"[INFO] Vertical line angle: {vertical_angle:.2f} degrees")

# ------ Step 5: Calculate relative angles with respect to the marker -------
relative_horizontal = horizontal_angle - marker_angle
relative_vertical = vertical_angle - marker_angle

print(f"[INFO] Relative Horizontal Angle: {relative_horizontal:.2f} degrees")
print(f"[INFO] Relative Vertical Angle: {relative_vertical:.2f} degrees")

# Optionally, visualize the lines over the circles
vis = image.copy()
for pt in horizontal_points:
    cv2.circle(vis, (int(pt[0]), int(pt[1])), 5, (0, 255, 0), -1)
for pt in vertical_points:
    cv2.circle(vis, (int(pt[0]), int(pt[1])), 5, (255, 0, 0), -1)
cv2.imshow("Central Points", vis)
cv2.waitKey(0)
cv2.destroyAllWindows()


[INFO] Loading image...
[INFO] Detected 1 marker(s) using DICT_ARUCO_ORIGINAL
[INFO] Marker angle: -88.06 degrees
[INFO] Found 5 points for the horizontal line and 3 for the vertical line.
[INFO] Horizontal line angle: 0.49 degrees
[INFO] Vertical line angle: 89.47 degrees
[INFO] Relative Horizontal Angle: 88.54 degrees
[INFO] Relative Vertical Angle: 177.53 degrees


In [2]:
import cv2
import numpy as np
import math
from sklearn.cluster import DBSCAN


# Helper Functions
def compute_marker_angle(marker_corners):
    """Compute marker orientation from its corners."""
    top_left = marker_corners[0]
    top_right = marker_corners[1]

    # Extract scalar values to avoid deprecation warning
    y_diff = float(top_right[1] - top_left[1])
    x_diff = float(top_right[0] - top_left[0])

    angle = math.degrees(math.atan2(y_diff, x_diff))
    return angle


def create_marker_mask(image, marker_corners, padding=10):
    """Create a mask that excludes the ArUco marker area."""
    mask = np.ones_like(image) * 255

    if marker_corners is not None and len(marker_corners) > 0:
        # Get the bounding rectangle of the marker
        x_min = int(max(0, np.min(marker_corners[:, 0]) - padding))
        y_min = int(max(0, np.min(marker_corners[:, 1]) - padding))
        x_max = int(min(image.shape[1], np.max(marker_corners[:, 0]) + padding))
        y_max = int(min(image.shape[0], np.max(marker_corners[:, 1]) + padding))

        # Create a black rectangle in the mask for the marker area
        cv2.rectangle(mask, (x_min, y_min), (x_max, y_max), 0, -1)

    return mask


def detect_colored_circles(image, marker_corners=None):
    """Detect circles in different color channels while excluding ArUco marker."""
    # Split the image into color channels
    b, g, r = cv2.split(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Create mask to exclude the ArUco marker area
    if marker_corners is not None:
        mask = create_marker_mask(gray, marker_corners)
        gray_masked = cv2.bitwise_and(gray, mask)
        b_masked = cv2.bitwise_and(b, mask)
        g_masked = cv2.bitwise_and(g, mask)
        r_masked = cv2.bitwise_and(r, mask)
    else:
        gray_masked = gray
        b_masked = b
        g_masked = g
        r_masked = r

    # Enhance images for better detection
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    gray_enhanced = clahe.apply(gray_masked)

    # Set up blob detector parameters - more sensitive settings
    params = cv2.SimpleBlobDetector_Params()
    #
    params.filterByArea = True
    params.minArea = 50  # Adjust depending on the expected size of circles
    params.maxArea = 5000  # Adjust as needed
    params.filterByCircularity = True
    params.minCircularity = 0.7  # Tighter circularity
    params.filterByInertia = True
    params.minInertiaRatio = 0.5

    # Create detector
    detector = cv2.SimpleBlobDetector_create(params)

    # Detect circles in multiple ways to catch more
    all_points = []

    # Try different channels and preprocessing
    for img in [gray_masked, gray_enhanced, b_masked, g_masked, r_masked]:
        keypoints = detector.detect(img)
        if keypoints:
            all_points.extend([kp.pt for kp in keypoints])

    # Remove duplicates
    if all_points:
        # Convert to numpy array for easier processing
        points_array = np.array(all_points)

        # Use DBSCAN to cluster very close points
        if len(points_array) > 1:
            clustering = DBSCAN(eps=10, min_samples=1).fit(points_array)
            labels = clustering.labels_

            # Keep only one point per cluster (the centroid)
            unique_labels = np.unique(labels)
            unique_points = []

            for label in unique_labels:
                cluster_points = points_array[labels == label]
                centroid = np.mean(cluster_points, axis=0)
                unique_points.append(centroid)

            unique_points = np.array(unique_points)
        else:
            unique_points = points_array
    else:
        unique_points = np.empty((0, 2))

    # Draw detected circles on visualization image
    vis_image = image.copy()
    for pt in unique_points:
        cv2.circle(vis_image, (int(pt[0]), int(pt[1])), 5, (0, 255, 0), 2)

    cv2.imshow("All Detected Circles", vis_image)
    cv2.waitKey(0)
    cv2.destroyWindow("All Detected Circles")

    return unique_points


def grid_clustering(points, axis=0, eps=10):
    """
    Cluster points into grid lines along specified axis.
    axis=0 for columns (x-coordinates), axis=1 for rows (y-coordinates).
    Returns clusters sorted by position.
    """
    if len(points) < 4:  # Need enough points for meaningful clustering
        return []

    # Extract the coordinates for the specified axis
    coords = points[:, axis].reshape(-1, 1)

    # Use DBSCAN to cluster points
    clustering = DBSCAN(eps=eps, min_samples=5).fit(coords)
    labels = clustering.labels_

    # Get unique labels, ignoring noise points (-1)
    unique_labels = np.unique(labels)
    unique_labels = unique_labels[unique_labels != -1]

    # Group points by cluster
    clusters = []
    for label in unique_labels:
        cluster_points = points[labels == label]
        # Calculate center position for sorting
        center_pos = np.mean(cluster_points[:, axis])
        clusters.append((center_pos, cluster_points))

    # Sort clusters by position
    clusters.sort(key=lambda x: x[0])

    return [cluster[1] for cluster in clusters]


def identify_central_lines(points, expected_rows=5, expected_cols=5):
    """
    Identify horizontal and vertical central lines from grid of points.
    Also attempts to interpolate missing points.
    """
    # Try different epsilon values if needed
    rows = grid_clustering(points, axis=1)
    columns = grid_clustering(points, axis=0)

    if not rows or not columns:
        return None, None

    # Find middle row and column
    middle_row_idx = len(rows) // 2
    middle_col_idx = len(columns) // 2

    if middle_row_idx < len(rows) and middle_col_idx < len(columns):
        return rows[middle_row_idx], columns[middle_col_idx]
    else:
        return None, None


def fit_line_angle(points):
    """Fit line to points and return angle in degrees."""
    if len(points) < 2:
        return None

    points = points.astype(np.float32)
    [vx, vy, x0, y0] = cv2.fitLine(points, cv2.DIST_L2, 0, 0.01, 0.01)

    # Extract scalar values from the arrays
    vx_scalar = float(vx[0])
    vy_scalar = float(vy[0])

    # Calculate angle in degrees
    angle = math.degrees(math.atan2(vy_scalar, vx_scalar))
    return angle


# Main Script
print("[INFO] Loading image...")
image_path = "DamasconeA.png"  # Change to your actual image path
image = cv2.imread(image_path)

if image is None:
    print(f"[ERROR] Could not read image from {image_path}")
    exit()

# Step 1: Detect ArUco Marker
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_ARUCO_ORIGINAL)
arucoParams = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(aruco_dict, arucoParams)
(corners, ids, rejected) = detector.detectMarkers(image)

marker_angle = None
marker_corners = None
if len(corners) > 0:
    print(f"[INFO] Detected {len(corners)} marker(s) using DICT_ARUCO_ORIGINAL")

    # Use the first marker detected
    marker_corners = corners[0][0]
    marker_angle = compute_marker_angle(marker_corners)
    print(f"[INFO] Marker angle: {marker_angle:.2f} degrees")

    # Draw detected marker
    marker_vis = image.copy()
    cv2.aruco.drawDetectedMarkers(marker_vis, corners, ids)
    cv2.imshow("Detected Markers", marker_vis)
    cv2.waitKey(0)
    cv2.destroyWindow("Detected Markers")
else:
    print("[ERROR] No ArUco markers detected!")
    exit()

# Step 2: Detect the circles (explicitly excluding the marker area)
all_circles = detect_colored_circles(image, marker_corners)
if len(all_circles) == 0:
    print("[ERROR] No circles detected!")
    exit()

print(f"[INFO] Detected {len(all_circles)} circles.")

# Step 3: Identify central horizontal and vertical lines
horizontal_points, vertical_points = identify_central_lines(all_circles)

if horizontal_points is None or vertical_points is None:
    print("[ERROR] Could not identify central lines.")
    exit()

print(f"[INFO] Found {len(horizontal_points)} points for horizontal line and {len(vertical_points)} for vertical line.")

# Step 4: Fit lines and compute angles
horizontal_angle = fit_line_angle(horizontal_points)

# For vertical line, use the same function but adjust for vertical reference
vertical_angle = fit_line_angle(vertical_points)
# Convert to angle from vertical reference (90 degrees offset)
if vertical_angle is not None:
    # Adjust to get the angle with respect to vertical (y-axis)
    vertical_angle = vertical_angle - 90 if vertical_angle >= 0 else vertical_angle + 90

if horizontal_angle is None or vertical_angle is None:
    print("[ERROR] Not enough points to compute line angles.")
    exit()

print(f"[INFO] Horizontal line angle: {horizontal_angle:.2f} degrees")
print(f"[INFO] Vertical line angle: {vertical_angle:.2f} degrees")

# Step 5: Calculate relative angles
if marker_angle is not None:
    # Calculate relative angles
    relative_horizontal = horizontal_angle - marker_angle
    relative_vertical = vertical_angle - marker_angle

    # Normalize angles to -90 to 90 degrees range
    if relative_horizontal > 90: relative_horizontal -= 180
    if relative_horizontal < -90: relative_horizontal += 180
    if relative_vertical > 90: relative_vertical -= 180
    if relative_vertical < -90: relative_vertical += 180

    print(f"[INFO] Relative Horizontal Angle: {relative_horizontal:.2f} degrees")
    print(f"[INFO] Relative Vertical Angle: {relative_vertical:.2f} degrees")

# Visualize the central lines and detected points
vis = image.copy()

# Draw all detected points
for pt in all_circles:
    cv2.circle(vis, (int(pt[0]), int(pt[1])), 3, (128, 128, 128), -1)

# Draw horizontal line points
for pt in horizontal_points:
    cv2.circle(vis, (int(pt[0]), int(pt[1])), 5, (0, 255, 0), -1)

# Draw vertical line points
for pt in vertical_points:
    cv2.circle(vis, (int(pt[0]), int(pt[1])), 5, (255, 0, 0), -1)

# Draw fitted lines
if horizontal_angle is not None:
    # Calculate centroid of horizontal points
    h_centroid = np.mean(horizontal_points, axis=0)
    # Use the angle to compute line direction
    h_dx = 1000 * math.cos(math.radians(horizontal_angle))
    h_dy = 1000 * math.sin(math.radians(horizontal_angle))
    # Draw the line through the centroid
    pt1 = (int(h_centroid[0] - h_dx), int(h_centroid[1] - h_dy))
    pt2 = (int(h_centroid[0] + h_dx), int(h_centroid[1] + h_dy))
    cv2.line(vis, pt1, pt2, (0, 255, 0), 2)

if vertical_angle is not None:
    # For vertical, we need to convert the angle back
    v_angle_rad = math.radians(vertical_angle + 90)
    # Calculate centroid of vertical points
    v_centroid = np.mean(vertical_points, axis=0)
    # Use the angle to compute line direction
    v_dx = 1000 * math.cos(v_angle_rad)
    v_dy = 1000 * math.sin(v_angle_rad)
    # Draw the line through the centroid
    pt1 = (int(v_centroid[0] - v_dx), int(v_centroid[1] - v_dy))
    pt2 = (int(v_centroid[0] + v_dx), int(v_centroid[1] + v_dy))
    cv2.line(vis, pt1, pt2, (255, 0, 0), 2)

cv2.imshow("Central Lines", vis)
cv2.waitKey(0)
cv2.destroyAllWindows()

[INFO] Loading image...
[INFO] Detected 1 marker(s) using DICT_ARUCO_ORIGINAL
[INFO] Marker angle: -88.06 degrees
[INFO] Detected 20 circles.
[ERROR] Could not identify central lines.


TypeError: object of type 'NoneType' has no len()

In [5]:
import cv2
import numpy as np
import math

# ----- Helper Functions -----
def compute_marker_angle(marker_corners):
    """
    Compute the marker orientation from its corners.
    We assume the order: top-left, top-right, bottom-right, bottom-left.
    Using the line from top-left to top-right, compute the angle in degrees.
    """
    top_left = marker_corners[0]
    top_right = marker_corners[1]
    angle = math.degrees(math.atan2(top_right[1] - top_left[1], top_right[0] - top_left[0]))
    return angle

def detect_grid_area(image, marker_corners):
    """
    Use the marker position to estimate where the grid is and crop the image accordingly.
    Return the cropped image.
    """
    # Get marker position and size
    marker_center = np.mean(marker_corners, axis=0)
    marker_size = np.max(marker_corners, axis=0) - np.min(marker_corners, axis=0)

    # Estimate grid position relative to marker
    # Assuming the grid is to the right and below the marker
    img_h, img_w = image.shape[:2]

    # Calculate crop coordinates
    # These values may need adjustment based on your specific setup
    crop_left = int(max(0, marker_center[0] - marker_size[0] * 0.5))
    crop_right = img_w
    crop_top = int(max(0, marker_center[1] - marker_size[1] * 0.5))
    crop_bottom = img_h

    # Crop the image
    cropped = image[crop_top:crop_bottom, crop_left:crop_right]

    print(f"[INFO] Cropped image from original {image.shape[:2]} to {cropped.shape[:2]}")

    return cropped, (crop_left, crop_top)

def upscale_image(image, scale_factor=2.0):
    """
    Upscale the image using cv2.resize with interpolation.
    """
    # Calculate new dimensions
    new_width = int(image.shape[1] * scale_factor)
    new_height = int(image.shape[0] * scale_factor)

    # Resize image
    upscaled = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_CUBIC)

    print(f"[INFO] Upscaled image from {image.shape[:2]} to {upscaled.shape[:2]}")

    return upscaled

def enhance_image(image):
    """
    Apply various image enhancement techniques to make circle detection easier.
    """
    # Convert to LAB color space for better color separation
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l, a, b = cv2.split(lab)
    l = clahe.apply(l)

    # Merge the CLAHE enhanced L-channel back with the a,b channels
    lab = cv2.merge((l, a, b))

    # Convert back to BGR
    enhanced = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

    # Apply slight Gaussian blur to reduce noise
    enhanced = cv2.GaussianBlur(enhanced, (3, 3), 0)

    return enhanced

def detect_circles_color_based(image):
    """
    Detect circles based on color segmentation of blue, red, and yellow.
    Returns coordinates of detected circles by color.
    """
    # Convert to HSV for easier color segmentation
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define color ranges
    color_ranges = {
        'blue': {'lower': np.array([90, 50, 50]), 'upper': np.array([130, 255, 255])},
        'red1': {'lower': np.array([0, 50, 50]), 'upper': np.array([10, 255, 255])},
        'red2': {'lower': np.array([170, 50, 50]), 'upper': np.array([180, 255, 255])},
        'yellow': {'lower': np.array([20, 50, 50]), 'upper': np.array([40, 255, 255])}
    }

    # Dictionary to store detected circles by color
    circles_by_color = {}

    # Process each color
    for color_name, color_range in color_ranges.items():
        # Create color mask
        mask = cv2.inRange(hsv, color_range['lower'], color_range['upper'])

        # Apply morphological operations to remove noise
        kernel = np.ones((5, 5), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

        # Show the mask for debugging
        cv2.imshow(f"{color_name} Mask", mask)
        cv2.waitKey(100)

        # Detect circles using Hough Circles on the mask
        circles = cv2.HoughCircles(
            mask, cv2.HOUGH_GRADIENT, dp=1, minDist=20,
            param1=50, param2=10, minRadius=5, maxRadius=30
        )

        # Store detected circles
        if circles is not None:
            # Convert to (x, y, radius) format
            circles = np.round(circles[0, :]).astype(int)

            # Store only (x, y) coordinates
            circle_coords = circles[:, :2]

            # Special handling for red (combine red1 and red2)
            if color_name == 'red1':
                circles_by_color['red'] = circle_coords
            elif color_name == 'red2' and 'red' in circles_by_color:
                circles_by_color['red'] = np.vstack([circles_by_color['red'], circle_coords])
            elif color_name == 'red2':
                circles_by_color['red'] = circle_coords
            else:
                circles_by_color[color_name] = circle_coords

            # Visualize detected circles
            vis = image.copy()
            for (x, y, r) in circles:
                cv2.circle(vis, (x, y), r, (0, 255, 0), 2)
            cv2.imshow(f"{color_name} Circles", vis)
            cv2.waitKey(100)

    return circles_by_color

def fit_line_angle(points):
    """Fit line to points and return angle in degrees."""
    if len(points) < 2:
        return None

    points = points.astype(np.float32)
    [vx, vy, x0, y0] = cv2.fitLine(points, cv2.DIST_L2, 0, 0.01, 0.01)

    # Extract scalar values from the arrays
    vx_scalar = float(vx[0])
    vy_scalar = float(vy[0])

    # Calculate angle in degrees
    angle = math.degrees(math.atan2(vy_scalar, vx_scalar))
    return angle

def visualize_results(original_image, circles_by_color, marker_angle, crop_offset=(0, 0), scale_factor=1.0):
    """
    Visualize detected circles and their organization by color.
    Calculate and display angles relative to the marker.
    """
    # Create a copy of the original image for visualization
    vis = original_image.copy()

    # Define colors for visualization (BGR format)
    vis_colors = {
        'blue': (255, 0, 0),
        'red': (0, 0, 255),
        'yellow': (0, 255, 255)
    }

    # Combined points for horizontal and vertical analysis
    all_points = []

    # Draw each color group and collect points
    for color_name, points in circles_by_color.items():
        if len(points) == 0:
            continue

        # Adjust coordinates back to original image space
        adjusted_points = []
        for pt in points:
            # Adjust for crop and scale
            orig_x = int((pt[0] / scale_factor) + crop_offset[0])
            orig_y = int((pt[1] / scale_factor) + crop_offset[1])
            adjusted_points.append((orig_x, orig_y))
            all_points.append((orig_x, orig_y))

            # Draw circle
            cv2.circle(vis, (orig_x, orig_y), 10, vis_colors.get(color_name, (0, 255, 0)), 2)

        # Convert to numpy array
        adjusted_points = np.array(adjusted_points)

        # Fit line if enough points
        if len(adjusted_points) >= 2:
            angle = fit_line_angle(adjusted_points)
            if angle is not None:
                relative_angle = angle - marker_angle
                print(f"[INFO] {color_name.capitalize()} points angle: {angle:.2f}°, "
                      f"relative to marker: {relative_angle:.2f}°")

                # Draw line through points
                [vx, vy, x0, y0] = cv2.fitLine(adjusted_points.astype(np.float32), cv2.DIST_L2, 0, 0.01, 0.01)
                x0, y0 = int(x0[0]), int(y0[0])

                # Calculate line endpoints
                if abs(vy[0]) > abs(vx[0] * 0.1):  # Non-vertical line
                    lefty = int((-x0 * vy[0] / vx[0]) + y0)
                    righty = int(((vis.shape[1] - x0) * vy[0] / vx[0]) + y0)
                    cv2.line(vis, (0, lefty), (vis.shape[1]-1, righty), vis_colors.get(color_name, (0, 255, 0)), 2)
                else:  # Near-vertical line
                    topx = int((-y0 * vx[0] / vy[0]) + x0)
                    bottomx = int(((vis.shape[0] - y0) * vx[0] / vy[0]) + x0)
                    cv2.line(vis, (topx, 0), (bottomx, vis.shape[0]-1), vis_colors.get(color_name, (0, 255, 0)), 2)

    # Convert all points to numpy array for grid analysis
    all_points = np.array(all_points)

    # Identify rows and columns (if enough points detected)
    if len(all_points) >= 5:
        # Sort by y-coordinate (rows)
        y_sorted = np.argsort(all_points[:, 1])
        sorted_points = all_points[y_sorted]

        # Group points into rows
        row_groups = []
        current_row = [sorted_points[0]]

        for i in range(1, len(sorted_points)):
            if abs(sorted_points[i][1] - current_row[0][1]) < 15:  # Tolerance for row
                current_row.append(sorted_points[i])
            else:
                row_groups.append(np.array(current_row))
                current_row = [sorted_points[i]]

        if current_row:
            row_groups.append(np.array(current_row))

        # Find row with most points (middle row likely has most)
        if row_groups:
            largest_row = max(row_groups, key=len)

            # Calculate angle of largest row
            if len(largest_row) >= 2:
                row_angle = fit_line_angle(largest_row)
                if row_angle is not None:
                    relative_row_angle = row_angle - marker_angle
                    print(f"[INFO] Largest row angle: {row_angle:.2f}°, "
                          f"relative to marker: {relative_row_angle:.2f}°")

                    # Draw the largest row with a thicker line
                    [vx, vy, x0, y0] = cv2.fitLine(largest_row.astype(np.float32), cv2.DIST_L2, 0, 0.01, 0.01)
                    x0, y0 = int(x0[0]), int(y0[0])
                    lefty = int((-x0 * vy[0] / vx[0]) + y0)
                    righty = int(((vis.shape[1] - x0) * vy[0] / vx[0]) + y0)
                    cv2.line(vis, (0, lefty), (vis.shape[1]-1, righty), (0, 255, 0), 3)

                    # Label the angle
                    cv2.putText(vis, f"Row angle: {row_angle:.1f}°", (10, 30),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                    cv2.putText(vis, f"Relative to marker: {relative_row_angle:.1f}°",
                               (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Sort by x-coordinate (columns)
        x_sorted = np.argsort(all_points[:, 0])
        sorted_points = all_points[x_sorted]

        # Group points into columns
        col_groups = []
        current_col = [sorted_points[0]]

        for i in range(1, len(sorted_points)):
            if abs(sorted_points[i][0] - current_col[0][0]) < 15:  # Tolerance for column
                current_col.append(sorted_points[i])
            else:
                col_groups.append(np.array(current_col))
                current_col = [sorted_points[i]]

        if current_col:
            col_groups.append(np.array(current_col))

        # Find column with most points
        if col_groups:
            largest_col = max(col_groups, key=len)

            # Calculate angle of largest column
            if len(largest_col) >= 2:
                col_angle = fit_line_angle(largest_col)
                if col_angle is not None:
                    relative_col_angle = col_angle - marker_angle
                    print(f"[INFO] Largest column angle: {col_angle:.2f}°, "
                          f"relative to marker: {relative_col_angle:.2f}°")

                    # Draw the largest column with a thicker line
                    [vx, vy, x0, y0] = cv2.fitLine(largest_col.astype(np.float32), cv2.DIST_L2, 0, 0.01, 0.01)
                    x0, y0 = int(x0[0]), int(y0[0])
                    topx = int((-y0 * vx[0] / vy[0]) + x0)
                    bottomx = int(((vis.shape[0] - y0) * vx[0] / vy[0]) + x0)
                    cv2.line(vis, (topx, 0), (bottomx, vis.shape[0]-1), (255, 0, 0), 3)

                    # Label the angle
                    cv2.putText(vis, f"Column angle: {col_angle:.1f}°", (10, 90),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
                    cv2.putText(vis, f"Relative to marker: {relative_col_angle:.1f}°",
                               (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    cv2.imshow("Final Results", vis)
    cv2.waitKey(0)

    return vis

# ----- Main Script -----
print("[INFO] Loading image...")
image_path = "DamasconeA.png"
image = cv2.imread(image_path)

if image is None:
    print(f"[ERROR] Could not read image from {image_path}")
    exit()

# Show original image
cv2.imshow("Original Image", image)
cv2.waitKey(0)

# ------ Step 1: Detect ArUco Marker ------
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_ARUCO_ORIGINAL)
arucoParams = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(aruco_dict, arucoParams)
(corners, ids, rejected) = detector.detectMarkers(image)

if len(corners) > 0:
    print(f"[INFO] Detected {len(corners)} marker(s) using DICT_ARUCO_ORIGINAL")

    # For demonstration we use the first marker detected
    marker_corners = corners[0][0]
    marker_angle = compute_marker_angle(marker_corners)
    print(f"[INFO] Marker angle: {marker_angle:.2f} degrees")

    # Draw the detected marker for visualization
    output = image.copy()
    cv2.aruco.drawDetectedMarkers(output, corners, ids)
    cv2.imshow("Detected Markers", output)
    cv2.waitKey(0)

else:
    print("[ERROR] No ArUco markers detected!")
    exit()

# ------ Step 2: Crop the image to focus on the grid area ------
cropped_image, crop_offset = detect_grid_area(image, marker_corners)
cv2.imshow("Cropped Image", cropped_image)
cv2.waitKey(0)

# ------ Step 3: Upscale the image ------
scale_factor = 2.0
upscaled_image = upscale_image(cropped_image, scale_factor)
cv2.imshow("Upscaled Image", upscaled_image)
cv2.waitKey(0)

# ------ Step 4: Enhance the image ------
enhanced_image = enhance_image(upscaled_image)
cv2.imshow("Enhanced Image", enhanced_image)
cv2.waitKey(0)

# ------ Step 5: Detect circles in the enhanced image ------
circles_by_color = detect_circles_color_based(enhanced_image)

# Print results
for color, circles in circles_by_color.items():
    print(f"[INFO] Detected {len(circles)} {color} circles")

# ------ Step 6: Visualize results and calculate angles ------
vis_result = visualize_results(image, circles_by_color, marker_angle, crop_offset, scale_factor)

cv2.destroyAllWindows()

[INFO] Loading image...
[INFO] Detected 1 marker(s) using DICT_ARUCO_ORIGINAL
[INFO] Marker angle: -88.06 degrees
[INFO] Cropped image from original (801, 1000) to (742, 903)
[INFO] Upscaled image from (742, 903) to (1484, 1806)
[INFO] Detected 1 blue circles
[INFO] Detected 2 red circles
[INFO] Red points angle: 2.33°, relative to marker: 90.39°
